# Framework de Preparação Assimétrica - Quick Start

**Autor:** Gabriel W. Soares  
**Objetivo:** Demonstração rápida do framework usando dados processados

---

## 📋 Pré-requisitos

Antes de executar este notebook, certifique-se de:

1. Ter executado o pipeline:
```bash
python run_pipeline.py
```

2. Ter as dependências instaladas:
```bash
pip install -r requirements.txt
pip install jupyter  # Se ainda não tiver
```

## 1. Importar Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 2. Carregar Dados Processados

In [2]:
# Verificar se dados existem
import os

arquivos_necessarios = [
    'data/processed/stress_index.csv',
    'data/processed/exposure_proxy.csv',
    'data/processed/defensive_concentration.csv',
    'data/processed/synchronization_index.csv'
]

for arquivo in arquivos_necessarios:
    if os.path.exists(arquivo):
        print(f"✅ {arquivo}")
    else:
        print(f"❌ {arquivo} NÃO ENCONTRADO!")
        print("   Execute: python run_pipeline.py")

❌ data/processed/stress_index.csv NÃO ENCONTRADO!
   Execute: python run_pipeline.py
❌ data/processed/exposure_proxy.csv NÃO ENCONTRADO!
   Execute: python run_pipeline.py
❌ data/processed/defensive_concentration.csv NÃO ENCONTRADO!
   Execute: python run_pipeline.py
❌ data/processed/synchronization_index.csv NÃO ENCONTRADO!
   Execute: python run_pipeline.py


In [3]:
# Carregar dados
stress = pd.read_csv('data/processed/stress_index.csv', index_col=0, parse_dates=True)
exposure = pd.read_csv('data/processed/exposure_proxy.csv', index_col=0, parse_dates=True)
defensive = pd.read_csv('data/processed/defensive_concentration.csv', index_col=0, parse_dates=True)
sync = pd.read_csv('data/processed/synchronization_index.csv', index_col=0, parse_dates=True)

print(f"Período de análise: {stress.index[0].date()} até {stress.index[-1].date()}")
print(f"Total de observações: {len(stress)} dias")

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/stress_index.csv'

## 3. Visualização Rápida: Índice de Estresse

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(stress.index, stress['STRESS_INDEX'], linewidth=2, color='red', label='Índice de Estresse')
ax.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)
ax.axhline(y=2, color='r', linestyle='--', linewidth=0.8, alpha=0.3, label='Limiar Crítico (2σ)')

# Destacar períodos de alto estresse
ax.fill_between(stress.index, 0, stress['STRESS_INDEX'], 
                where=(stress['STRESS_INDEX'] > 2), 
                alpha=0.3, color='red', label='Alto Estresse')

ax.set_xlabel('Data', fontsize=12)
ax.set_ylabel('Z-score', fontsize=12)
ax.set_title('Índice de Estresse Sistêmico', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Estatísticas do Índice:")
print(f"   Média: {stress['STRESS_INDEX'].mean():.3f}")
print(f"   Desvio: {stress['STRESS_INDEX'].std():.3f}")
print(f"   Máximo: {stress['STRESS_INDEX'].max():.3f} em {stress['STRESS_INDEX'].idxmax().date()}")
print(f"   Mínimo: {stress['STRESS_INDEX'].min():.3f} em {stress['STRESS_INDEX'].idxmin().date()}")

## 4. Análise de Exposição vs. Estresse

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Painel 1: Estresse
axes[0].plot(stress.index, stress['STRESS_INDEX'], color='red', linewidth=2)
axes[0].axhline(y=2, color='r', linestyle='--', alpha=0.3)
axes[0].fill_between(stress.index, 0, stress['STRESS_INDEX'], 
                     where=(stress['STRESS_INDEX'] > 2), 
                     alpha=0.2, color='red')
axes[0].set_ylabel('Índice de Estresse', fontsize=11)
axes[0].set_title('Estresse Sistêmico', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Painel 2: Exposição
axes[1].plot(exposure.index, exposure['EXPOSURE_RATIO'], color='green', linewidth=2)
axes[1].set_ylabel('Ratio (Base 100)', fontsize=11)
axes[1].set_xlabel('Data', fontsize=11)
axes[1].set_title('Exposição Institucional (FXI/SPY)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🔍 Análise Visual:")
print("   Observe se quedas em Exposição precedem aumentos em Estresse")
print("   Isso seria consistente com preparação assimétrica")

## 5. Sincronização Temporal

In [ ]:
# Comparar sincronização em diferentes regimes
common_index = sync.index.intersection(stress.index)
sync_aligned = sync.loc[common_index, 'SYNC_INDEX']
stress_aligned = stress.loc[common_index, 'STRESS_INDEX']

# Definir regimes
high_stress = stress_aligned > stress_aligned.quantile(0.75)
low_stress = stress_aligned < stress_aligned.quantile(0.25)

sync_high = sync_aligned[high_stress]
sync_low = sync_aligned[low_stress]

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
data_to_plot = [sync_low.dropna(), sync_high.dropna()]
bp = axes[0].boxplot(data_to_plot, labels=['Baixo Estresse', 'Alto Estresse'], patch_artist=True)
bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('lightcoral')
axes[0].set_ylabel('Sincronização', fontsize=11)
axes[0].set_title('Distribuição por Regime de Estresse', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')

# Scatter plot
axes[1].scatter(stress_aligned, sync_aligned, alpha=0.5, s=30)
axes[1].set_xlabel('Estresse', fontsize=11)
axes[1].set_ylabel('Sincronização', fontsize=11)
axes[1].set_title('Relação: Estresse × Sincronização', fontsize=12, fontweight='bold')

# Linha de tendência
z = np.polyfit(stress_aligned.dropna(), sync_aligned.dropna(), 1)
p = np.poly1d(z)
axes[1].plot(sorted(stress_aligned.dropna()), p(sorted(stress_aligned.dropna())), 
            "r--", alpha=0.8, linewidth=2)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Teste estatístico
from scipy import stats
t_stat, p_value = stats.ttest_ind(sync_high.dropna(), sync_low.dropna())

print(f"\n📊 Teste t para diferença de médias:")
print(f"   Sincronização (Baixo Estresse): {sync_low.mean():.3f}")
print(f"   Sincronização (Alto Estresse):  {sync_high.mean():.3f}")
print(f"   t-statistic: {t_stat:.3f}")
print(f"   p-value: {p_value:.4f}")

if p_value < 0.05:
    print(f"   ✅ Diferença SIGNIFICANTE (p<0.05)")
    print(f"   → Sincronização aumenta durante crises")
else:
    print(f"   ❌ Diferença NÃO significante")

## 6. Análise Customizada: Seu Turno!

Use as células abaixo para suas próprias análises:

In [ ]:
# Exemplo: Calcular correlação entre variáveis
dados_combinados = pd.concat([
    stress['STRESS_INDEX'],
    exposure['EXPOSURE_RATIO'],
    sync['SYNC_INDEX']
], axis=1).dropna()

correlacao = dados_combinados.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlacao, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', square=True, linewidths=1)
plt.title('Matriz de Correlação', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Sua análise aqui!
# Ideias:
# - Análise de subperíodos
# - Correlação rolling
# - Identificação de eventos específicos
# - Comparação com outros indicadores

## 7. Exportar Resultados

Salve suas análises customizadas:

In [ ]:
# Exemplo: Exportar estatísticas descritivas
estatisticas = pd.DataFrame({
    'Estresse': stress['STRESS_INDEX'].describe(),
    'Exposição': exposure['EXPOSURE_RATIO'].describe(),
    'Sincronização': sync['SYNC_INDEX'].describe()
})

estatisticas.to_csv('output/estatisticas_descritivas.csv')
print("✅ Estatísticas salvas em: output/estatisticas_descritivas.csv")

estatisticas

---

## 📚 Próximos Passos

1. **Aprofunde-se na teoria:** Leia [FRAMEWORK.md](../FRAMEWORK.md)
2. **Customize análises:** Veja [IMPLEMENTATION_GUIDE.md](../IMPLEMENTATION_GUIDE.md)
3. **Execute testes estatísticos:** Use `scripts/02_leadlag_analysis.py` como referência
4. **Replique para outros contextos:** Adapte ETFs e período

---

**Autor:** Gabriel W. Soares  
**Versão:** 1.0  
**Licença:** MIT